<a href="https://colab.research.google.com/github/MustafizSaadi/DMLab/blob/main/Decision_Tree_numeric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import gzip
from io import StringIO
import re
import sys
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
# cd /content/drive/My\ Drive

In [ ]:
class tree_node:
  def __init__(self, attribute, value, index):
    self.attribute = attribute
    self.value = value
    self.dictionary = dict()
    self.label = dict()
    self.child = []
    self.attribute_index = index

    self.dictionary["less"] = 0
    self.dictionary["great"] = 1

  def add_child(self, node, label, order, type):
    self.child.append(node)
    if type == "empty":
        self.label[self.dictionary[order]] = label


  # def add_child(self, label, val):
  #   self.child.append(None)
  #   self.label[self.dictionary[val]] = label



In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
def parse_csv(csv_str, columns, full_table, labels):
  i = 0
  tuples = csv_str.split('\n')
  for s in tuples:
    data = s.split(',')
    data = list(filter(None, data))
    # data = data[1:]
    if i == 0:
      for d in data[:-1]:
        columns.append(d)
    elif len(data)!=0:
      labels.append(data[len(data)-1])
      data = [float(d) for d in data[:-1]]
      full_table.append(data)
    i += 1
  print(columns)
  print(full_table)
  print(labels)
  return columns, full_table, labels

In [ ]:
def parse_csv2(csv_str):
  attribute_dictionary = dict()
  tuples = csv_str.split('\n')
  for s in tuples:
    data = s.split(',')
    attribute = data[0]
    if attribute != '' :
      attribute_dictionary[attribute] = data[1:]
  return attribute_dictionary

In [ ]:
def load_dataset_wine(file_name):
  # attribute_dictionary = dict()
  df = pd.read_csv(file_name)
  fixed_columns = ['class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium',
	                'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity',
 	                'Hue', 'OD280/OD315 of diluted wines', 'Proline' ]
  # df.to_csv(file_name, header = fixed_columns, index=False)

  # df = pd.read_csv(file_name)
  labels = df['class'].values.tolist()
  fixed_columns.remove('class')
  df = df[fixed_columns]

  # for column in fixed_columns:
  #   attribute_dictionary[column] = df[column].unique()
  #   # print(len(attribute_dictionary[column]))
  # attribute_dictionary['class'] = np.unique(np.array(labels))

  full_table = df.values.tolist()
  # fixed_columns.append('class')
  # print(labels)
  return fixed_columns, full_table, labels


In [ ]:
def load_dataset_ILPD(file_name):
  # attribute_dictionary = dict()
  df = pd.read_csv(file_name)
  fixed_columns = ['age', 'gender', 'total Bilirubin', 'direct Bilirubin', 'total proteins', 'albumin', 'A/G ratio', 'SGPT', 'SGOT', 'Alkphos', 'class' ]
  # df.to_csv(file_name, header = fixed_columns, index=False)

  # df = pd.read_csv(file_name)
  labels = df['class'].values.tolist()
  fixed_columns.remove('class')
  df['gender'] = df['gender'].replace('Female', 1)
  # df['gender'] = df['gender'].replace('Male', 0)
  df = df[fixed_columns]
  print(df['gender'])

  # for column in fixed_columns:
  #   attribute_dictionary[column] = df[column].unique()
  #   # print(len(attribute_dictionary[column]))
  # attribute_dictionary['class'] = np.unique(np.array(labels))

  full_table = df.values.tolist()
  # fixed_columns.append('class')
  # print(labels)
  return fixed_columns, full_table, labels

In [ ]:
def info(database, indices, labels):
  answer = 0
  cnt = dict()
  for ind in indices:
    tup = database[ind]
    label = labels[ind]
    if label in cnt:
      cnt[label] += 1
    else:
      cnt[label] = 1
  
  for label in cnt:
    if cnt[label] == len(indices):
      answer += 0
    else:
      answer += (cnt[label]/len(indices)) * math.log2(cnt[label]/len(indices))

  answer *= -1

  # print(answer)

  return answer


In [ ]:
def get_smaller_database(database, labels, splitting_attribute_index, indices):
  new_database = []
  new_labels = []

  for ind in indices:
    # new_tup = []
    new_tup = database[ind]
    # for i in range(len(tup)):
    #   if i != splitting_attribute_index:
    #     new_tup.append(tup[i])
    new_database.append(new_tup)
    new_labels.append(labels[ind])
  return new_database, new_labels

In [ ]:
def majority_for_no_attribute(columns, database, labels):
  number_of_attributes = len(columns)
  if number_of_attributes == 0:
    label_dict = dict()
    max_label = ""
    cnt = -1
    for label in labels:
      if label not in label_dict:
        label_dict[label] = 1
      else:
        label_dict[label] += 1

    for label in label_dict:
      if label_dict[label] > cnt:
        cnt = label_dict[label]
        max_label = label

    return True, max_label
  else:
    return False, None

In [ ]:
def majority_class(labels):
  label_dict = dict()
  max_label = ""
  cnt = -1
  for label in labels:
    if label not in label_dict:
      label_dict[label] = 1
    else:
      label_dict[label] += 1

  for label in label_dict:
    if label_dict[label] > cnt:
      cnt = label_dict[label]
      max_label = label

  return max_label

In [ ]:
def create_decision_tree(columns, database, labels):
  print(len(database))
  print("database ", database)
  number_of_attributes = len(columns)
  min = sys.maxsize
  splitting_attribute = ""
  splitting_attribute_index = -1
  split_point = -1
  
  for i in range(number_of_attributes):
    min_info_attribute = sys.maxsize
    best_mid_point = -1
    # database = sorted(database, key=lambda x: x[i])
    singleColumn = [data[i] for data in database]
    singleColum = sorted(singleColumn)
    # print("start*****")
    # print(i)
    # print(database)
    for j in range(len(database)-1):
      # print(database[j][i])
      info_attribute = 0
      mid_point = (singleColum[j] + singleColum[j+1])/2
      # print("mid point", mid_point)
      indices = [ind for ind in range(len(database)) if database[ind][i] <= mid_point]
      # if mid_point == 6.95 :
      #   print("leftIndex = {}".format(indices))

      # print(indices)
      # if(len(indices) != 0):
      # print("lesseqal = {}".format((len(indices)/len(database)) * info(database, indices, labels)))
      info_attribute += (len(indices)/len(database)) * info(database, indices, labels)

      indices = [ind for ind in range(len(database)) if database[ind][i] > mid_point]
      # if mid_point == 6.95 :
      #   print("rightIndex = {}".format(indices))
      # # print(indices)
      # # if(len(indices) != 0):
      # print("greater = {}".format((len(indices)/len(database)) * info(database, indices, labels)))
      info_attribute += (len(indices)/len(database)) * info(database, indices, labels)

      # print("info_attribute = {}".format(info_attribute))

      if info_attribute < min_info_attribute:
        min_info_attribute = info_attribute
        best_mid_point = mid_point
        # print("best_mid_point in j", best_mid_point)
      
    # print("min_info_attribute = {}".format(min_info_attribute))
    if min_info_attribute < min:
      min = min_info_attribute
      splitting_attribute = columns[i]
      splitting_attribute_index = i
      split_point = best_mid_point
      # print("best_mid_point in i", best_mid_point)

  # columns.remove(splitting_attribute)

  # new_columns = []

  # for i in range(number_of_attributes):
  #   if i != splitting_attribute_index:
  #     new_columns.append(columns[i])
  
  # print("splitting_attribute_index = {}".format(splitting_attribute_index))
  # print("splitting_attribute = {}".format(splitting_attribute))
  # print("split_point = {}".format(split_point))
  # return None

  node = tree_node(splitting_attribute, split_point, splitting_attribute_index)
  label = majority_class(labels)

  indices = [ind for ind in range(len(database)) if database[ind][splitting_attribute_index] <= split_point]
  # print(indices)
  if(len(indices) != 0):
    if info(database, indices, labels) == 0:
      node.add_child(None, labels[indices[0]], "less", "empty")
    else:
      new_database, new_labels = get_smaller_database(database, labels, splitting_attribute_index, indices)
      if(len(new_database) == len(database)):
        node.add_child(None, labels[indices[0]], "less", "empty")
      else:
        child_node = create_decision_tree(columns, new_database, new_labels)
        node.add_child(child_node, None, None, "")
  else:
    print("majority_class_needed")
    node.add_child(None, label, "less", "empty")
    # return None

    

  indices = [ind for ind in range(len(database)) if database[ind][splitting_attribute_index] > split_point]
  # print(indices)
  if(len(indices) != 0):
    if info(database, indices, labels) == 0:
      node.add_child(None, labels[indices[0]], "great", "empty")
    else:
      new_database, new_labels = get_smaller_database(database, labels, splitting_attribute_index, indices)
      if(len(new_database) == len(database)):
        node.add_child(None, labels[indices[0]], "great", "empty")
      else:
        child_node = create_decision_tree(columns, new_database, new_labels)
        node.add_child(child_node, None, None, "")
  else:
    print("majority_class_needed")
    node.add_child(None, label, "great", "empty")
    # return None
    
  return node



  



    #  = distinct(columns[:][i])

In [ ]:
def print_path(node):
  print(node.attribute)
  print(node.value)

  for i in range(len(node.child)):
    child = node.child[i]
    if child == None:
      print(node.label[i])
    else:
      print_path(child)
    


In [ ]:
def predict(node, test):
  ind = node.attribute_index
  # print(ind)
  val = test[ind]
  # print(val)
  if val <= node.value:
    val_index = 0
  else:
    val_index = 1

  if node.child[val_index] == None:
    return node.label[val_index]
  else:
    # think about majority voting
    # test.remove(val)
    return predict(node.child[val_index], test)

In [ ]:
# f = open('iris.data', 'r')
# columns, database, labels = parse_csv(f.read(-1), [], [], [])
# # print(columns)
# X_train, X_test, y_train, y_test = train_test_split(database, labels, test_size=0.2)
# print(X_train)
# print(y_train)
num_epochs = 10
accuracy = 0
precision = 0
recall = 0

for i in range(num_epochs):
  # f = open('winequality-red.csv', 'r')
  # f = open('glass.data', 'r')
  # columns, database, labels = parse_csv(f.read(-1), [], [], [])
  # print(columns)
  # columns, database, labels = load_dataset_wine('wine.data')
  # f = open('Wholesale customers data.csv', 'r')
  # columns, database, labels = parse_csv(f.read(-1), [], [], [])
  # columns, database, labels = load_dataset_ILPD('Indian Liver Patient Dataset (ILPD).csv')

  f = open('haberman.data', 'r')
  columns, database, labels = parse_csv(f.read(-1), [], [], [])
  X_train, X_test, y_train, y_test = train_test_split(database, labels, test_size=0.2)
  print(len(X_train), len(X_test), len(y_train), len(y_test))
  root_node = create_decision_tree(columns, X_train, y_train)
  prediction = []
  for test in X_test:
    # print("size ", len(test))
    # print(test)
    output = predict(root_node, test)
    if output == None :
      # print(test)
      print("None Type object found")
    else:
      prediction.append(output)
  print(prediction)
  accuracy += accuracy_score(y_test, prediction)
  precision += precision_score(y_test, prediction, average = 'macro')
  recall += recall_score(y_test, prediction, average = 'macro')

print("accuracy ", accuracy/num_epochs)
print("precision ", precision/num_epochs)
print("recall ", recall/num_epochs)
# f = open('attribute_values.csv', 'r')
# attribute_dictionary = parse_csv2(f.read(-1))
# print(attribute_dictionary)

['age', 'year', 'positive_nodes']
[[30.0, 64.0, 1.0], [30.0, 62.0, 3.0], [30.0, 65.0, 0.0], [31.0, 59.0, 2.0], [31.0, 65.0, 4.0], [33.0, 58.0, 10.0], [33.0, 60.0, 0.0], [34.0, 59.0, 0.0], [34.0, 66.0, 9.0], [34.0, 58.0, 30.0], [34.0, 60.0, 1.0], [34.0, 61.0, 10.0], [34.0, 67.0, 7.0], [34.0, 60.0, 0.0], [35.0, 64.0, 13.0], [35.0, 63.0, 0.0], [36.0, 60.0, 1.0], [36.0, 69.0, 0.0], [37.0, 60.0, 0.0], [37.0, 63.0, 0.0], [37.0, 58.0, 0.0], [37.0, 59.0, 6.0], [37.0, 60.0, 15.0], [37.0, 63.0, 0.0], [38.0, 69.0, 21.0], [38.0, 59.0, 2.0], [38.0, 60.0, 0.0], [38.0, 60.0, 0.0], [38.0, 62.0, 3.0], [38.0, 64.0, 1.0], [38.0, 66.0, 0.0], [38.0, 66.0, 11.0], [38.0, 60.0, 1.0], [38.0, 67.0, 5.0], [39.0, 66.0, 0.0], [39.0, 63.0, 0.0], [39.0, 67.0, 0.0], [39.0, 58.0, 0.0], [39.0, 59.0, 2.0], [39.0, 63.0, 4.0], [40.0, 58.0, 2.0], [40.0, 58.0, 0.0], [40.0, 65.0, 0.0], [41.0, 60.0, 23.0], [41.0, 64.0, 0.0], [41.0, 67.0, 0.0], [41.0, 58.0, 0.0], [41.0, 59.0, 8.0], [41.0, 59.0, 0.0], [41.0, 64.0, 0.0], [41.0, 

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
root_node = create_decision_tree(columns, X_train, y_train)

244
database  [[50.0, 59.0, 2.0], [46.0, 62.0, 5.0], [53.0, 58.0, 1.0], [72.0, 64.0, 0.0], [38.0, 69.0, 21.0], [44.0, 58.0, 9.0], [55.0, 58.0, 1.0], [34.0, 61.0, 10.0], [67.0, 66.0, 0.0], [60.0, 67.0, 2.0], [46.0, 65.0, 20.0], [52.0, 60.0, 5.0], [52.0, 68.0, 0.0], [47.0, 65.0, 0.0], [57.0, 62.0, 14.0], [58.0, 61.0, 2.0], [63.0, 63.0, 0.0], [74.0, 63.0, 0.0], [46.0, 62.0, 0.0], [41.0, 58.0, 0.0], [41.0, 60.0, 23.0], [66.0, 61.0, 13.0], [48.0, 58.0, 11.0], [31.0, 59.0, 2.0], [64.0, 66.0, 0.0], [47.0, 60.0, 4.0], [68.0, 68.0, 0.0], [67.0, 61.0, 0.0], [58.0, 58.0, 3.0], [54.0, 69.0, 7.0], [37.0, 63.0, 0.0], [41.0, 67.0, 0.0], [43.0, 64.0, 2.0], [53.0, 65.0, 12.0], [48.0, 66.0, 0.0], [64.0, 68.0, 0.0], [65.0, 67.0, 0.0], [62.0, 58.0, 0.0], [65.0, 58.0, 0.0], [37.0, 59.0, 6.0], [45.0, 66.0, 0.0], [47.0, 63.0, 6.0], [69.0, 66.0, 0.0], [46.0, 69.0, 3.0], [51.0, 64.0, 7.0], [52.0, 61.0, 0.0], [59.0, 64.0, 1.0], [61.0, 62.0, 5.0], [66.0, 68.0, 0.0], [50.0, 58.0, 1.0], [60.0, 61.0, 25.0], [54.0, 

In [ ]:
print_path(root_node)

positive_nodes
2.0
age
76.0
age
40.0
age
34.0
age
33.5
1
year
59.5
2
1
1
age
47.0
year
63.0
year
59.0
age
41.0
1
age
42.0
positive_nodes
1.0
2
1
2
1
year
68.5
year
67.0
positive_nodes
1.0
age
42.5
year
65.0
year
64.0
age
41.0
1
1
1
2
age
44.0
2
year
64.5
1
year
66.5
2
age
45.0
age
45.0
2
2
1
1
1
2
year
65.0
age
59.0
age
53.0
year
62.0
positive_nodes
1.0
1
age
51.0
1
age
52.5
2
1
age
52.0
age
50.0
age
48.5
1
age
49.5
2
year
63.5
1
age
50.0
2
2
1
2
1
age
60.5
2
year
60.5
year
59.0
year
58.0
age
63.0
2
age
64.5
1
age
71.0
age
66.0
positive_nodes
0.0
age
65.0
age
65.0
2
2
age
66.0
1
1
1
2
1
1
2
positive_nodes
0.0
age
61.0
year
64.5
1
2
age
70.5
1
age
72.0
year
63.5
2
1
1
2
year
67.0
1
positive_nodes
0.0
1
age
66.0
2
1
2
age
42.0
positive_nodes
20.5
year
64.5
1
age
34.0
year
66.5
2
1
1
age
36.0
1
2
positive_nodes
8.0
year
64.0
positive_nodes
6.0
age
50.5
age
46.0
positive_nodes
3.0
1
2
1
year
60.0
year
59.0
age
55.5
2
age
64.0
1
2
1
2
1
age
46.5
2
age
47.5
1
year
68.5
positive_nodes
4.0
age

In [ ]:
# test = [5.35,3.0,2.0,1.25]
# prediction = predict(root_node, test)
# print(prediction)
prediction = []
for test in X_test:
  print("size ", len(test))
  print(test)
  prediction.append(predict(root_node, test))

size  3
[62.0, 62.0, 6.0]
size  3
[44.0, 64.0, 6.0]
size  3
[77.0, 65.0, 3.0]
size  3
[31.0, 65.0, 4.0]
size  3
[67.0, 66.0, 0.0]
size  3
[36.0, 60.0, 1.0]
size  3
[38.0, 66.0, 0.0]
size  3
[61.0, 59.0, 0.0]
size  3
[54.0, 65.0, 5.0]
size  3
[43.0, 63.0, 14.0]
size  3
[58.0, 67.0, 0.0]
size  3
[43.0, 59.0, 2.0]
size  3
[64.0, 65.0, 22.0]
size  3
[43.0, 64.0, 0.0]
size  3
[35.0, 64.0, 13.0]
size  3
[37.0, 63.0, 0.0]
size  3
[49.0, 66.0, 0.0]
size  3
[45.0, 67.0, 0.0]
size  3
[60.0, 64.0, 0.0]
size  3
[60.0, 61.0, 1.0]
size  3
[47.0, 66.0, 0.0]
size  3
[70.0, 63.0, 0.0]
size  3
[41.0, 69.0, 8.0]
size  3
[67.0, 63.0, 1.0]
size  3
[53.0, 60.0, 9.0]
size  3
[40.0, 58.0, 2.0]
size  3
[38.0, 62.0, 3.0]
size  3
[58.0, 61.0, 1.0]
size  3
[57.0, 64.0, 0.0]
size  3
[62.0, 58.0, 0.0]
size  3
[67.0, 64.0, 8.0]
size  3
[34.0, 60.0, 1.0]
size  3
[59.0, 64.0, 4.0]
size  3
[61.0, 65.0, 8.0]
size  3
[57.0, 64.0, 1.0]
size  3
[47.0, 62.0, 0.0]
size  3
[52.0, 60.0, 4.0]
size  3
[62.0, 65.0, 19.0]
size  3


In [ ]:
print(accuracy_score(y_test, prediction))

0.5806451612903226


In [ ]:
print(precision_score(y_test, prediction, average = 'macro'))
print(recall_score(y_test, prediction, average = 'macro'))

0.5546218487394958
0.5773809523809523


In [ ]:
print(sys.maxsize)

9223372036854775807
